# Capítulo 08 — Explorações Extras e Reflexões

Este notebook acompanha o Capítulo 08 da série **Fazendo um LLM do Zero**.

Este notebook é um laboratório aberto.

🎯 **Objetivos deste notebook:**
- Limitações dos modelos didáticos
- Impacto do tamanho do dataset
- Comparações entre estratégias
- Geração multi-turno
- Experimentos livres


## 1. Setup e Configuração

In [ ]:
# ============================================================
# Setup do repositório
# ============================================================
import os

REPO_URL = "https://github.com/vongrossi/fazendo-um-llm-do-zero.git"
REPO_DIR = "fazendo-um-llm-do-zero"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

os.chdir(REPO_DIR)
print("Diretório atual:", os.getcwd())


### 1.1 Dependências e Imports

In [ ]:
!pip -q install -r 08-extras/requirements.txt

import torch
import random
import numpy as np
import matplotlib.pyplot as plt
import sys

# Adiciona raiz ao path
sys.path.append(os.getcwd())

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Carregar Modelo Instruction Tuned

In [ ]:
from lib.gptmini import GPTConfig, GPTMini

config = GPTConfig(
    vocab_size=2000,
    context_size=64,
    d_model=128,
    n_heads=4,
    n_layers=2
)

model = GPTMini(config).to(device)

try:
    model.load_state_dict(torch.load("07_instruction_gpt.pt", map_location=device))
    print("Modelo Instruction Tuned carregado")
except:
    print("Modelo não encontrado — use checkpoints anteriores")


### 2.1 Funções de Tokenização

In [ ]:
chars = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,!?\n:")
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for c,i in stoi.items()}

def encode(text):
    return [stoi[c] for c in text if c in stoi]

def decode(tokens):
    return "".join([itos[t] for t in tokens])


## 3. Experimento 1 — Impacto do Tamanho do Prompt

Aqui vamos observar como o tamanho do contexto influencia a resposta do modelo.

In [ ]:
@torch.no_grad()
def generate(model, prompt, max_tokens=80):

    tokens = encode(prompt)
    tokens = torch.tensor(tokens).unsqueeze(0).to(device)

    for _ in range(max_tokens):
        logits, _ = model(tokens)
        next_token = torch.argmax(logits[:, -1, :], dim=-1)
        tokens = torch.cat([tokens, next_token.unsqueeze(1)], dim=1)

    return decode(tokens.squeeze().tolist())


In [ ]:
prompt_curto = "Explique IA:"
prompt_longo = "Explique IA considerando aprendizado supervisionado e não supervisionado:"

print(generate(model, prompt_curto))
print("\n---\n")
print(generate(model, prompt_longo))


## 4. Experimento 2 — Limitação de Dataset Pequeno

Vamos observar comportamento fora do domínio treinado.

In [ ]:
print(generate(model, "Explique física quântica:"))


## 5. Experimento 3 — Simulando Conversa Multi-turno

Aqui simulamos uma interação contínua com o modelo.

In [ ]:
conversation = """
Usuário: O que é Machine Learning?
Assistente:
"""

print(generate(model, conversation))


## 6. Experimento 4 — Comparar Temperatura

Criatividade vs Determinismo

In [ ]:
def generate_temp(model, prompt, temperature=1.0):

    tokens = encode(prompt)
    tokens = torch.tensor(tokens).unsqueeze(0).to(device)

    for _ in range(80):
        logits, _ = model(tokens)

        probs = torch.softmax(logits[:, -1, :] / temperature, dim=-1)
        next_token = torch.multinomial(probs, 1)

        tokens = torch.cat([tokens, next_token], dim=1)

    return decode(tokens.squeeze().tolist())


In [ ]:
print(generate_temp(model, "Explique IA:", temperature=0.3))
print("\n---\n")
print(generate_temp(model, "Explique IA:", temperature=1.2))


## 7. Experimento 5 — Demonstrando Hallucination

In [ ]:
print(generate(model, "Quem descobriu a linguagem Python em 1800?"))


### 7.1 O Que Esses Experimentos Mostram

Modelos pequenos:

• dependem fortemente do dataset  
• podem produzir respostas plausíveis mas incorretas  
• possuem limitação de contexto  
• demonstram comportamento probabilístico  

Essas características também existem em modelos maiores, embora com menor frequência.


🧪 Experimento 6 — Prompt Engineering

In [ ]:
print(generate(model, "Responda como um professor explicando IA para iniciantes:"))


## 9. Visualizando Distribuição de Probabilidades

In [ ]:
@torch.no_grad()
def show_next_token_distribution(model, prompt):

    tokens = encode(prompt)
    tokens = torch.tensor(tokens).unsqueeze(0).to(device)

    logits, _ = model(tokens)
    probs = torch.softmax(logits[0, -1], dim=-1).cpu()

    plt.bar(range(len(probs)), probs)
    plt.title("Distribuição do Próximo Token")
    plt.show()


In [ ]:
show_next_token_distribution(model, "Explique IA:")


## 10. Reflexões Finais

Este notebook demonstrou que:

• Modelos didáticos possuem limitações claras  
• Compreender fundamentos permite interpretar comportamento  
• Pequenas mudanças em dados e prompts impactam resultados  
• LLMs são sistemas probabilísticos, não sistemas conscientes  

Este laboratório marca o encerramento da jornada prática da série.
